In [8]:
import pandas as pd
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS
from astropy import coordinates as coord
from astropy import units as u
import numpy as np

import warnings
warnings.filterwarnings("ignore") # This is mostly just aesthetic

pd.set_option('display.max_rows', None) # Makes pandas display the entire table rather than just the first and last few rows.
Simbad.add_votable_fields('otype') # The object type is not returned by default from SIMBAD so this adds the column to the returned table

In [9]:
# Reads in the data from the data files in the final project folder
CV_yes = pd.DataFrame(pd.read_csv("CV_Cayenne-problYESCOPY.txt", sep="\s+"))
CV_no = pd.DataFrame(pd.read_csv("CV_Cayenne-problNOCOPY.txt", sep="\s+"))

# Creates the new columns in the tables.
CV_yes['CV?'] = "Y" # Y means we think this is a CV
CV_yes['new_CV?'] = "" # Empty column to fill in below
CV_yes['otype'] = ""# Empty column to fill in later
CV_no['CV?'] = "N" # N means we think this is NOT a CV

The below cell will sort through all the objects in the list of suspected CV stars and will use their ra and dec to search SIMBAD and does one of two things:

* If the object does not exist in SIMBAD, it is automatically labeled as a new discovery.
* If the object is listed as 'candidate', it is also listed as new. All other objects are listed as not new. 

The object type is also taken from SIMBAD and entered into a new column in the table and then the cell underneath that is used to visualize all the types of objects there are thay may not fit into the other categories (nothing, known CV, or candidate CV).

In [10]:
YvN = []
otype = []

for x in range(0, len(CV_yes['ra'])):
    # This can be done with the CV_no table as well
    coords = coord.SkyCoord(ra=CV_yes['ra'][x], dec=CV_yes['dec'][x], unit=(u.deg, u.deg))
    
    result = Simbad.query_region(coords, radius=2*u.arcsec)
    
    # Makes list to label CV as new when no entry in SIMBAD is found
    YvN.append('Y' if result is None else 'N')
    
    # Makes a list to label a CV as 'Nothing' to the object type when no entry in SIMBAD is found
    otype.append('Nothing' if result is None else result['OTYPE'][0])
 
    
for j in range(0, len(YvN)):
    # Puts the previous two lists into the columns in the table
    CV_yes['new_CV?'][j] = YvN[j]
    CV_yes['otype'][j] = otype[j]

# NOTE: If using the CV_no table, comment out the following code

# Creates a list of indices corresponding to the objects labelled as candidate
candidate = np.where(CV_yes['otype']==b'Candidate_CV*')

for x in candidate:
    # Based on the indicies list, changes 'N' to 'Y' when the object type is candidate
    CV_yes['new_CV?'][x] = 'Y'

The below cell will count the total number of each classification of object so you can see any extra objects that may not just be CV or candidate. As you can see, it returns quite a few objects that are not previously know CVs, but are also one of a kind in this list and so much be changed by hand.

In [16]:
print(CV_yes['otype'].value_counts())

# CV_yes

Nothing               18
b'CataclyV*'          15
b'DwarfNova'          13
b'Candidate_CV*'       7
b'AMHer'               5
b'Candidate_RRLyr'     2
b'Blue'                1
b'EB*Algol'            1
b'EB*'                 1
Name: otype, dtype: int64


In [12]:
h = 1
"""
This is the part that does not work. As you can see, I have tried a number of different methods of pulling it up.
The second line defining sp is the way the website for SDSS suggests how to do it. xid is defined and runs and the
get_images one runs fine, but I get an error for all of the get_spectra lines.
"""
coords = coord.SkyCoord(ra=CV_yes['ra'][h], dec=CV_yes['dec'][h], unit=(u.deg, u.deg))

xid = SDSS.query_region(coords, spectro=True, data_release=15)

# sp = SDSS.get_spectra(plate=CV_yes['plate'][h], fiberID=CV_yes['fiber'][h], mjd=CV_yes['mjd'][h], data_release=15)
# sp = SDSS.get_spectra(matches = xid)
# sp = SDSS.get_spectra(plate=6284, fiberID=222, mjd=56310, radius=100)
# im = SDSS.get_images(matches = xid, band='g')


In [14]:
def spectra(h):
    """
    Returns the spectrum of a star in the table corresponding to index h (first row is h = 0).
    Currently does not work because I cannot run the "get_spectra" function for some reason.
    """
    coords = coord.SkyCoord(ra=CV_yes['ra'][h], dec=CV_yes['dec'][h], unit=(u.deg, u.deg))
    xid = SDSS.query_region(coords, spectro=True, data_release=15)
    sp = SDSS.get_spectra(matches = xid)
    
    return sp
